#### Challenge 49: CBC-MAC Message Forgery

[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)

In [1]:
from Crypto.Util import number
from Crypto.Random import random
from Crypto.Hash.SHA256 import SHA256Hash

import math
import base64
import cryptopals as cp

from decimal import *

import pdb

<div class="alert alert-block alert-info">   

Let's talk about CBC-MAC. 
    
CBC-MAC is like this: 
    
1. Take the plaintext `P`.
2. Encrypt `P` under CBC with key `K`, yielding ciphertext `C`.
3. Chuck all of `C` but the last block `C[n]`.
4. `C[n]` is the MAC.
    
</div>

In [2]:
# ---------UNKNOWN PARAMETERS ------------
K_Shared = random.Random.get_random_bytes(32)
# ---------END UNKNOWN PARAMETERS---------

IV = b'\x00' * 16
test_msg = b'Some Test Message That\'s Longer than one block'

In [3]:
def CBC_MAC(p, K, IV):
    
    a_out = cp.AESEncrypt(p, K, 'CBC', IV)
    return(a_out[-16:])

<div class="alert alert-block alert-info">   

Suppose there's an online banking application, and it carries out user requests by talking to an API server over the network. Each request looks like this: 

`message || IV || MAC`
    
The message looks like this: 
    
`from=#{from_id}&to=#{to_id}&amount=#{amount}`
    
Now, write an API server and a web frontend for it. (NOTE: No need to get ambitious and write actual servers and web apps. Totally fine to go lo-fi on this one.) The client and server should share a secret key `K` to sign and verify messages. 
    
The API server should accept messages, verify signatures, and carry out each transaction if the MAC is valid. It's also publicly exposed - the attacker can submit messages freely assuming he can forge the right MAC. 
    
The web client should allow the attacker to generate valid messages for accounts he controls. (Feel free to sanitize params if you're feeling anal-retentive.) Assume the attacker is in a position to capture and inspect messages from the client to the API server. 
    
</div>

---

The way I interpret this challenge, based on this first stage of the attack, only the first block of the message is being manipulated. Therefore, to get it to work you have to make some big assumptions:

1.  The `to_id` will have to appear in the first block (this means the account #'s have to be short enough for that to happen), and 
2.  We have intercepted a previous valid transaction from the target account owner to someone else _in the amount we want to steal_.  

---

In [4]:
class account:
    
    def __init__(self, starting_balance):
            
        #self.id = id
        self.balance = starting_balance
        self.key = random.Random.get_random_bytes(32)
        
    def __str__(self):
        
        display_str = ''
        display_str += f'Account Key:  {self.key}\n'
        display_str += f'Account Balance: ${self.balance:,.2f}\n'
        return (display_str)
        
class bank_app:

    next_id = 1
    
    def __init__(self):
        
        # Create some test accounts
        self.accounts = {}
        
        self.accounts[self.next_id] = account(5000000)
        self.next_id += 1
        self.accounts[self.next_id] = account(300000)
        self.next_id += 1
        self.accounts[self.next_id] = account(70230.24)
        self.next_id += 1
    
    def __str__(self):
        
        display_str = ''
        for account in self.accounts:
            display_str += 'Account ID:  ' + str(account) + '\n'
            display_str += str(self.accounts[account])
            display_str += '\n'
            
        return(display_str)
    
    def open_new_account(self, deposit_amount):
        
        id = self.next_id
        self.accounts[id] = account(deposit_amount)
        self.next_id += 1
        
        return(id)
    
    def process_msg(self, msg):
        
        if len(msg) < 32:
            raise(ValueError('Invalid message)'))
        
        IV, MAC = msg[-32:-16], msg[-16:]
        transaction = cp.strip_PKCS7_pad(msg[:-32]).split(b'&')
        
        if transaction[0][:5] == b'from=' and \
           transaction[1][:3] == b'to=' and \
           transaction[2][:7] == b'amount=':
            
            start = transaction[0].find(b'=') + 1
            from_id = int(transaction[0][start:])            
            key = self.accounts[from_id].key        
            myMAC = CBC_MAC(msg[:-32], key, IV)       
            
        else:
        
            raise(Exception('Invalid transaction format'))
                    
        if (MAC == myMAC):            
            
            start = transaction[1].find(b'=') + 1
            to_id = int(transaction[1][start:])            

            #print(transaction[2])
            start = transaction[2].find(b'=') + 1
            amount = float(transaction[2][start:])     

        
        if self.accounts[from_id].balance >= amount:
            self.accounts[from_id].balance -= amount
            self.accounts[to_id].balance += amount
            return(True)
        else:
            return(False)    

In [5]:
def create_signed_transaction(from_id, to_id, amount):
    
    # To simplify things, just assuming the authentic user has access to their account transaction key.
    id_key = my_bank.accounts[from_id].key
    msg = cp.PKCS7_pad((f'from={from_id}&to={to_id}&amount={amount}').encode())
    IV = b'\x00'*16
    MAC = CBC_MAC(msg, id_key, IV)
    
    return(msg + IV + MAC)
    

In [6]:
my_bank = bank_app()

In [14]:
print('*****************************')
print('Balances before transfer')
print('*****************************\n')
print(my_bank)

print('Transfering $1000000.00 from Acct 10001 to Acct 10002')
valid_transfer_1E6 = create_signed_transaction(1, 2, 1000000)
print()
print('*****************************')
print('Balances after transfer')
print('*****************************\n')
my_bank.process_msg(t)
print(my_bank)

*****************************
Balances before transfer
*****************************

Account ID:  1
Account Key:  b'OOf\x86\x10q\xf5\xbb:\xaf8\x1f\x83P\xd69\xd8\xa4k\x86\x8b-\xd5\xc0\x7f>\xb0\xb3\xa4n\xb58'
Account Balance: $3,000,000.00

Account ID:  2
Account Key:  b'\x9d\xf0{\x1e\tW\x10\xf4\x82\x1ccN?~R\x0f\xeb\\\x83\xc0\xb6\xfc\xdb\xefR\x10\x13M\xc2\xb0*\xda'
Account Balance: $1,300,000.00

Account ID:  3
Account Key:  b"r\x10\x04\x07\xc6\xc9\xd0_\x1a\xc7\xec\x16\r\xd1\x03db*\xdf`\x08~\xbc\xda\xb8\xe6\xfc'|\xee[\n"
Account Balance: $70,230.24

Account ID:  4
Account Key:  b'i\xaa\xc5\r\x90S\xae>\xe3\x8a\x93\x82\x87\xb1\xb9P\xdf\xa8\xf7;\xb6\x93\x8c\n\r\xa4\xd2\xd9\x99\xe5\xd9\xed'
Account Balance: $1,000,000.00


Transfering $1000000.00 from Acct 10001 to Acct 10002

*****************************
Balances after transfer
*****************************

Account ID:  1
Account Key:  b'OOf\x86\x10q\xf5\xbb:\xaf8\x1f\x83P\xd69\xd8\xa4k\x86\x8b-\xd5\xc0\x7f>\xb0\xb3\xa4n\xb58'
Account Ba

<div class="alert alert-block alert-info">   

One thing we haven't discussed is the `IV`. Assume the client generates a per-message IV and sends it along with the MAC. That's how CBC works, right? 
Wrong. 
    
For messages signed under CBC-MAC, an attacker-controlled `IV` is a liability. Why? Because it yields full control over the first block of the message. 
Use this fact to generate a message transferring 1M spacebucks from a target victim's account into your account. 
    
I'll wait. Just let me know when you're done. 
... waiting 
... waiting 
... waiting 
All done? Great - I knew you could do it! 

</div>

In [8]:
# Open a new $0 balance account that I control and have the key for. 
my_id = my_bank.open_new_account(0)
my_key = my_bank.accounts[my_id].key

In [20]:
# Eve watched and captured earlier transfer of $1,000,000 from Acct 1 to Acct 2:
intercepted_msg = valid_transfer_1E6

malicious_message = f'from=1&to={my_id}&amou'.encode()
intercepted_IV = intercepted_msg[-32:-16]
forged_IV = cp.bitwise_xor(cp.bitwise_xor(intercepted_msg[0:16], intercepted_IV), malicious_message)
my_evil_transaction = malicious_message + intercepted_msg[16:-32] + \
                      forged_IV + intercepted_msg[-16:]   
print(f"Eve's Balance before attack: ${my_bank.accounts[my_id].balance:,.2f}")
my_bank.process_msg(my_evil_transaction)
print(f"Eve's Balance after attack: ${my_bank.accounts[my_id].balance:,.2f}")

Eve's Balance before attack: $3,000,000.00
Eve's Balance after attack: $3,000,000.00


<div class="alert alert-block alert-info">   

Now let's tune up that protocol a little bit. 
    
As we now know, you're supposed to use a fixed `IV` with CBC-MAC, so let's do that. We'll set ours at `0` for simplicity. This means the `IV` comes out of the protocol: 
    
`message || MAC`
    
Pretty simple, but we'll also adjust the message. For the purposes of efficiency, the bank wants to be able to process multiple transactions in a single request. So the message now looks like this: 
    
`from=#{from_id}&tx_list=#{transactions}`
    
With the transaction list formatted like: 
    
`to:amount(;to:amount)*`

</div>

In [21]:
class new_bank(bank_app):
    
    def process_msg(self, msg):
                
        if len(msg) < 32:
            raise(ValueError('Invalid message)'))
        
        IV = b'\x00' * 16
        MAC = msg[-16:]
        data = cp.strip_PKCS7_pad(msg[:-16])
        transaction = data.split(b'&')
        
        if transaction[0][:5] == b'from=' and \
           transaction[1][:8] == b'tx_list=':
            
            start = transaction[0].find(b'=') + 1
            from_id = int(transaction[0][start:])            
            key = self.accounts[from_id].key       
            #pdb.set_trace()
            myMAC = CBC_MAC(msg[:-16], key, IV)       
            
        else:
        
            raise(Exception('Invalid transaction format'))
                    
        if (MAC != myMAC):            
            return(False)
        
        start = transaction[1].find(b'=') + 1
        transaction_list = transaction[1][start:].split(b';')           

        for transfer in transaction_list:
            to_id, amount = transfer.split(b':')
            to_id = int(to_id)
            amount = float(amount)
            if self.accounts[from_id].balance >= amount:
                self.accounts[from_id].balance -= amount
                self.accounts[to_id].balance += amount
            else:
                return(False)    
        
        return(True)
def new_create_signed_transaction(from_id, transaction_list):
    
    # To simplify things, just assuming the authentic user has access to their account transaction key.
    id_key = my_new_bank.accounts[from_id].key
    
    # Formats the message per part 2 of Challenge #49.
    # Transfer list is a list of tuples, with element [0] of each pair being the "to_id" and element
    # [1] being the "amount"
    
    msg = f'from={from_id}&tx_list='
    for transfer in transaction_list:
        msg += f'{transfer[0]}:{transfer[1]};'
    msg = cp.PKCS7_pad(msg[:-1].encode())
    IV = b'\x00'*16
    MAC = CBC_MAC(msg, id_key, IV)
    
    return(msg + MAC)

In [23]:
my_new_bank = new_bank()
print(my_new_bank)

# Test some transfers 
print('Transfering funds\n\nNew Balances: \n')
from_id = 1
transfer_list = [[2, 1000000],[3, 1001.99]]
transaction = new_create_signed_transaction(from_id, transfer_list)
my_new_bank.process_msg(transaction)

print(my_new_bank)

Account ID:  1
Account Key:  b'\x11/\xddD{\xa7\xe4w\xe75\x0f\xa2\xa8@\xfd&\xee\x07Z\x13\xd8\xfb\xbc\xb2^8\x02\xf5z\xad\xf7\x83'
Account Balance: $5,000,000.00

Account ID:  2
Account Key:  b'\x99E\xc0\xc7z\xbf\x9f\x8c\x80\x8d\xa1\xb7\x18\x01a\x8d\xe2\x8dL\xe9X\x9f\xe0\xed\xebSS\xef\nCa\xeb'
Account Balance: $300,000.00

Account ID:  3
Account Key:  b'\x1f\xdf3\xfc\xcd\xe9\x98gd\x0f\x19\xf9Kz\x97\xa1| ;\xa5\xf0q\xcfn3\x82\x9ff?\xc4\xaa\x9f'
Account Balance: $70,230.24


Transfering funds

New Balances: 

Account ID:  1
Account Key:  b'\x11/\xddD{\xa7\xe4w\xe75\x0f\xa2\xa8@\xfd&\xee\x07Z\x13\xd8\xfb\xbc\xb2^8\x02\xf5z\xad\xf7\x83'
Account Balance: $3,998,998.01

Account ID:  2
Account Key:  b'\x99E\xc0\xc7z\xbf\x9f\x8c\x80\x8d\xa1\xb7\x18\x01a\x8d\xe2\x8dL\xe9X\x9f\xe0\xed\xebSS\xef\nCa\xeb'
Account Balance: $1,300,000.00

Account ID:  3
Account Key:  b'\x1f\xdf3\xfc\xcd\xe9\x98gd\x0f\x19\xf9Kz\x97\xa1| ;\xa5\xf0q\xcfn3\x82\x9ff?\xc4\xaa\x9f'
Account Balance: $71,232.23




<div class="alert alert-block alert-info">   

There's still a weakness here: the MAC is vulnerable to length extension attacks. How? 
    
Well, the output of CBC-MAC is a valid `IV` for a new message. 
    
_"But we don't control the `IV` anymore!"_
    
With sufficient mastery of CBC, we can fake it. 
    
Your mission: capture a valid message from your target user. Use length extension to add a transaction paying the attacker's account 1M spacebucks. 

<div class="alert alert-block alert-warning">    

#### **Hint!**
    
This would be a lot easier if you had full control over the first block of your message, huh? Maybe you can simulate that. 
    
</div>
    
_Food for thought: How would you modify the protocol to prevent this?_
    
</div>    

In [24]:
# Open a new $0 balance account that I control and have the key for. 
attacker_id = my_new_bank.open_new_account(0)

# Eve watched and captured earlier transfer of $1,000,000 from Acct 1 to Acct 2:
intercepted_msg = bytearray(transaction)

# Here's what I need to add onto a transaction while still producing a valid MAC:
transfer_to_me = f';{attacker_id}:1000000'

### I'm stuck...what next?  

Have to do length extension while still producing a valid MAC.  A valid IV for part 1 isn't useful without also knowing the resulting MAC.  We don't know the MAC for additional blocks beyond this, so how does knowing it help to do length extension?

If this was CBC-MAC using a hash function, knowing the output would allow continuing indefinitely..

Very stuck here.  

Some data that might help?


Here's a [good paper](https://cseweb.ucsd.edu/~mihir/papers/cbc.pdf) on CBC security in general.


From [Wikipedia](https://en.wikipedia.org/wiki/CBC-MAC#Using_predictable_initialization_vector):

> If the block cipher used is secure (meaning that it is a pseudorandom permutation), then CBC-MAC is secure for fixed-length messages. However, by itself, it is not secure for variable-length messages. Thus, any single key must only be used for messages of a fixed and known length. This is because an attacker who knows the correct message-tag (i.e. CBC-MAC) pairs for two messages `(m, t)` and `(m′ , t′)` can generate a third message `m″` whose CBC-MAC will also be `t′`. This is simply done by XORing the first block of `m′` with `t` and then concatenating m with this modified `m′`; i.e., by making `m″ = m ‖ [ (m1′ ⊕ t) ‖ m2′ ‖ … ‖ mx′ ]`. When computing the MAC for the message `m″`, it follows that we compute the MAC for `m` in the usual manner as `t`, but when this value is chained forwards to the stage computing `E_K_MAC (m1′ ⊕ t)` we will perform an exclusive OR operation with the value derived for the MAC of the first message. The presence of that tag in the new message means it will cancel, leaving no contribution to the MAC from the blocks of plain text in the first message `m: E_ K_MAC (m1′ ⊕ t ⊕ t ) = E_K_MAC (m1′)` and thus the tag for `m″` is `t′`. 

This observation looks like the key tidbit:

> **The presence of that tag in the new message means it will cancel, leaving no contribution to the MAC from the blocks of plain text in the first message**

---

Let's try some forgeries on CBC-MAC to get a feel for things:

In [10]:
m0 = cp.PKCS7_pad(b'Here is a message with some words in it')
t0 = cp.CBC_MAC(m0, key, IV, False)
print(t0.hex())

m1 = cp.PKCS7_pad(b'Here is another message with more words')
t1 = cp.CBC_MAC(m1, key, IV, False)
print(t1.hex())

m0_m1 = m0[:-16] + cp.bitwise_xor(m0[-16:], m1[:16]) + m1[16:]
print("\nHere's the chained message:\n")
print(m0_m1)
t01 = cp.CBC_MAC(m1, key, IV, False)
print()
print(t01.hex())

assert(t01 == t1)

596520c8f49d672f5148b1c86bb36eb3
08837aea3c062f4564b7e03e30b1ddb6

Here's the chained message:

b'Here is a message with some word;E\x1b\x0b\x00\x00\x07)hgf}al{)message with more words\t\t\t\t\t\t\t\t\t'

08837aea3c062f4564b7e03e30b1ddb6


---
**Observe:**   We had to corrupt an intermediate block, but were able to concatenate two messages together and still generate a valid MAC.   

`CBC_MAC(m1) == CBC_MAC(m0_m1)`

---

Let's try some simple forgeries from [this 2004 paper](https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=4&cad=rja&uact=8&ved=2ahUKEwi2ot2H8NboAhWXB50JHYlWCIIQFjADegQIBBAB&url=https%3A%2F%2Fwww.cosic.esat.kuleuven.be%2Fpublications%2Farticle-61.pdf&usg=AOvVaw2rQ9pI4IDkVPcZCle0av1d) by Preneel:

---

**1. Given**

`MAC(x)`

one knows that 

`MAC( x || (x ⊕ MAC(x) ) = MAC(x)` 

for a *single block* `x`

![Image](images/CBC-MACForgery_1.svg)

In [26]:
x = cp.PKCS7_pad(b'Test1')
t = cp.CBC_MAC(x, key, IV, False)

x_ = x + cp.bitwise_xor(x, t)
t_ = cp.CBC_MAC(x_, key, IV, False)

print('\nCase 1:\n')
print(t.hex())
print(t_.hex())
assert(t==t_)


Case 1:

9c5e8579c7341ff74644f26e7ccecb95
9c5e8579c7341ff74644f26e7ccecb95


---

**2. Given**

- `MAC(x)` 
- `MAC(x')` 

one knows that 

`MAC( x || (x' ⊕ MAC(x) ) = MAC(x')`

![Case 2](images/CBC-MACForgery_2.svg)

In [28]:
x = cp.PKCS7_pad(b'Test1')
t = cp.CBC_MAC(x, key, IV, False)

x_ = cp.PKCS7_pad(b'Message 2')
t_ = cp.CBC_MAC(x_, key, IV, False)

x0_ = x + cp.bitwise_xor(x_, t)
t0_ = cp.CBC_MAC(x0_, key, IV, False)
print('\nCase 2:\n')
print(t.hex())
print(t_.hex())
print(t0_.hex())
assert(t_ == t0_)


Case 2:

9c5e8579c7341ff74644f26e7ccecb95
eaf3198f2a89d406c95e68c5d160a4fd
eaf3198f2a89d406c95e68c5d160a4fd


---

**3. Given:**

- `MAC(x), MAC(x||y)`, and 
- `MAC(x')`, 

one knows that 

`MAC(x' || y') = MAC(x || y)` 

if 

`y' = y ⊕ MAC(x) ⊕ MAC(x')`


In [29]:
x = cp.PKCS7_pad(b'Test1')
MAC_x = cp.CBC_MAC(x, key, IV, False)

x_ = cp.PKCS7_pad(b'Test2')
MAC_x_ = cp.CBC_MAC(x_, key, IV, False)

y = cp.PKCS7_pad(b'Test3')
MAC_y = cp.CBC_MAC(y, key, IV, False)

y_ = cp.bitwise_xor(cp.bitwise_xor(y, MAC_x), MAC_x_)
MAC_x_y_ = cp.CBC_MAC(x_ + y_, key, IV, False)
MACxy = cp.CBC_MAC(x + y, key, IV, False)

print('\nCase 3\n')
print(MAC_x_y_.hex())
print(MACxy.hex())
assert(MACxy == MAC_x_y_)


Case 3

f98cabc4a5efff16d26a5821edf10989
f98cabc4a5efff16d26a5821edf10989


---

> If `x` and `x'` have a common sequence of s trailing blocks and if the compression function `f` is a permutation (for fixed `x_i`), the collision must occur at `H_(t-s)`, i.e. just before the common blocks. After deleting the `s` common blocks in `x` and `x'`, one still has an internal collision.   In this case the attack can be enhanced since this provides additional freedom in the choice of the forged text by Lemma 1. In particular, if z and 2' have the same length one can obtain a forgery on a text of that length. See **[paper by Preneel](file:///C:/Users/cobb/Documents/GitHub/References/PO97%20-%20MDx-MAC%20and%20Building%20MACs%20from%20Hash%20Functions.pdf)**

[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)